## Librairies

In [1]:
from gymnasium import Env
import gymnasium
import numpy as np
import os
from stable_baselines3.common import env_checker
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
from gymnasium.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete
from cargame import Car, Track, Vector
import pygame
import sys


## Load Environment

In [59]:
tt = Dict({
            'Grid' : Box(low = 0, high = 255, shape = (96, 96)),
            'CarAngle' : Box(low = 0, high = 1, shape = (1,1)),
            'TireAngle' : Box(low = -1, high = 1, shape = (1,1))
        })

speed_size = 1
angle = 1

# ttttt = np.concatenate([i.low.flatten() for i in tt.values()])
# print(ttttt.sample)
te = Box(
    low = np.concatenate([
        tt['Grid'].low.flatten(),
        tt['CarAngle'].low.flatten()
    ]),
    high = np.concatenate([
        tt['Grid'].high.flatten(),
        tt['CarAngle'].high.flatten()
    ]),
    shape = (speed_size + angle + 96*96 -1,)
)

print(te.sample())

[2.2773973e+02 2.2283751e+02 2.3580814e+02 ... 7.3112976e+01 1.8350497e+02
 5.2181561e-02]


In [8]:
from time import time

class CarEnvironment(Env):
    MAX_EPOCH_TIME = 3 * 60

    def __init__(self,
                 ):
        self.action_space = Box(low=np.array([0, -1.0]), high=np.array([1.0, 1.0]), shape = (2,), dtype = np.float32)

        print("Loading pixels from image")
        pixels = image.transform_image('imageprocessing/image_test/test3redgreen.png')
        print(pixels.shape)
        plt.imshow(pixels, cmap='gray') 

        print("Creating track")
        track = Track(pixels)
        self.timestep = 0.1
        
        print("Creating Car")
        self.car = Car(12, 15, 2, 30, track, 0.1, Vector(0, 7), Vector(4.5, -5))

        print("Making observation space")
        cm_per_pixel = self.car.body_length / track.car_dimensions.y
        self.observation_space = Dict({
            'GridR' : Box(low = 0, high = 255, shape = (pixels.shape[1], pixels.shape[0]), dtype=np.uint8),
            'GridG' : Box(low = 0, high = 255, shape = (pixels.shape[1], pixels.shape[0]), dtype=np.uint8),
            'GridB' : Box(low = 0, high = 255, shape = (pixels.shape[1], pixels.shape[0]), dtype=np.uint8),
            'CarPositionX' : Box(low = 0, high = pixels.shape[1] * cm_per_pixel, shape=(1,1)),
            'CarPositionY' : Box(low = 0, high = pixels.shape[0] * cm_per_pixel, shape=(1,1)),
            'CarSpeed' : Box(low = 0, high = 1, shape = (1,1)),
            'CarAngle' : Box(low = 0, high = 1, shape = (1,1)),
            'TireAngle' : Box(low = -1, high = 1, shape = (1,1))
        })

        grid_size = self.observation_space['Grid'].shape[0] * self.observation_space['Grid'].shape[1] 
        one_dimension_size = 5
        
        print("Making flat observation space")
        self.observation_space = Box(
            low = np.concatenate([
                self.observation_space['Grid'].low.flatten(),
                self.observation_space['CarPositionX'].low.flatten(), 
                self.observation_space['CarPositionY'].low.flatten(),
                self.observation_space['CarSpeed'].low.flatten(),
                self.observation_space['CarAngle'].low.flatten(), 
                self.observation_space['TireAngle'].low.flatten() 
            ]),
                
            high = np.concatenate([
                self.observation_space['GridR'].high.flatten(), 
                self.observation_space['GridG'].high.flatten(), 
                self.observation_space['GridB'].high.flatten(), 
                self.observation_space['CarPositionX'].low.flatten(),
                self.observation_space['CarPositionY'].low.flatten(),
                self.observation_space['CarSpeed'].high.flatten(), 
                self.observation_space['CarAngle'].high.flatten(), 
                self.observation_space['TireAngle'].high.flatten()                            
            ]),
            shape = (grid_size + one_dimension_size,)
        )
        pixels = image.transform_image('imageprocessing/image_test/real_test.jpg')
        track = Track(pixels)
        self.timestep = 0.1
        self.car = Car(10, 10, 10, 30, track, 0.1, Vector(0, 5), Vector(0, -5))
        self.start_time = time()
    
    def step(self, action):
        print("Reward:", self.car.reward)
        # Apply Action 
        angle = action[1]
        speed = action[0]

        # Action
        self.car.set_front_tire_angle(angle)
        self.car.set_speed(speed)
        self.car.update()
        
        # Terminated
        truncated = (time() - self.start_time) > CarEnvironment.MAX_EPOCH_TIME
        return self.car.get_state(), self.car.reward, self.car.check_if_done(), self.car.check_if_terminated(), truncated
    
    def reset(self):
        self.car.reset()
        self.start_time = time()

        return (self.car.get_state(), {})
    
    def render(self):
        screen = pygame.display.set_mode((4000,3000))
        bg_track = pygame.image.load('atlas/imageprocessing/image_test/test3redgreen.png')

        while True:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    sys.exit()

    def close(self) -> None:
        pass

In [30]:
env = CarEnvironment()
env.render()

KeyError: 'Grid'

## Train RL Model

In [9]:
env = CarEnvironment()
env = DummyVecEnv([lambda : env])

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [4]:
env_checker.check_env(env, warn=True, skip_render_check=True)

/Users/jeffbrin/.local/lib/python3.12/site-packages/stable_baselines3/common/env_checker.py:461: UserWarning: We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) cf. https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html
  warnings.warn(


Reward: 5
True
Reward: 5
True
Reward: 5
True
Reward: 5
True
Reward: 5
True
Reward: 5
True
Reward: 5
True
Reward: 5
True
Reward: 5
True
Reward: 5
True
Reward: 5
True


In [8]:
log_path = os.path.join('Training', 'Logs')
# model = PPO('CnnPolicy', env, verbose = 1, tensorboard_log=log_path)
model = PPO('MlpPolicy', env, verbose = 2, tensorboard_log=log_path, n_steps=2)


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/Users/jeffbrin/.local/lib/python3.12/site-packages/stable_baselines3/ppo/ppo.py:155: UserWarning: You have specified a mini-batch size of 64, but because the `RolloutBuffer` is of size `n_steps * n_envs = 2`, after every 0 untruncated mini-batches, there will be a truncated mini-batch of size 2
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=2 and n_envs=1)
  warnings.warn(


In [14]:
env.reset()
model.learn(total_timesteps=50, progress_bar=True, )


Logging to Training/Logs/PPO_31


Output()

Reward: 5

True

Reward: 5

...

Reward: 5

True

Reward: 5

True

--------------------------------------
| rollout/                |          |
|    ep_len_mean          | 1        |
|    ep_rew_mean          | 4        |
| time/                   |          |
|    fps                  | 19       |
|    iterations           | 3        |
|    time_elapsed         | 0        |
|    total_timesteps      | 6        |
| train/                  |          |
|    approx_kl            | 0.0      |
|    clip_fraction        | 0        |
|    clip_range           | 0.2      |
|    entropy_loss         | -2.84    |
|    explained_variance   | nan      |
|    learning_rate        | 0.0003   |
|    loss                 | 0        |
|    n_updates            | 640      |
|    policy_gradient_loss | 0        |
|    std                  | 1        |
|    value_loss           | 0        |
--------------------------------------


## Save Model

In [33]:
import pygame

In [ ]:
screen = pygame.display.set_mode((4000,3000))
bg_track = pygame.image.load('imageprocessing/image_test/test3redgreen.png')

while True:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()